In [57]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.metrics import confusion_matrix


# If you have these qualifications, is it more worth to further education or more experience for each level?

In [58]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
data = pd.read_csv('Levels_Fyi_Salary_Data.csv')
amazon = data[data['company'] == 'Amazon']
amazon = amazon.replace({'level':{'SDE I':'L4','SDE II':'L5','SDE III':'L6','L6 SDM':'L6','6':'L6','L5 SDM':'L5','Principal SDE':'L7','Senior SDE':'L6','5':'L5','7':'L7','4':'L4','1':'L1','l5':'L5','l4':'L4','l6':'L6','l7':'L7','l1':'L1','l2':'L2','l3':'L3','2':'L2','3':'L3'}})
cleaned = amazon.loc[(amazon['level'] == 'L1') | (amazon['level']=='L2') | (amazon['level']=='L3') | (amazon['level']=='L4') | (amazon['level']=='L5') | (amazon['level']=='L6') | (amazon['level']=='L7') | (amazon['level']=='L8')]
cleaned = cleaned.sort_values(by=['level','totalyearlycompensation'], ascending=[False,False])

# from here onwards only add items

In [59]:
print("PermaHeadDamage:\n",data['Doctorate_Degree'].value_counts())
print("Masters:\n",data['Masters_Degree'].value_counts())
print("Bachelors:\n",data['Bachelors_Degree'].value_counts())
print("Highschool:\n",data['Highschool'].value_counts())
print("education:\n",data['Education'].value_counts())
fillededu = data[data['Education'].notnull()]
#fillededu.info()
#fillededu['company'].value_counts()
top5 = fillededu[fillededu['company'].isin(['Facebook','Google','Amazon','Apple','Microsoft'])]
#top5.info()
top5['company'].value_counts()

PermaHeadDamage:
 0    60839
1     1803
Name: Doctorate_Degree, dtype: int64
Masters:
 0    47251
1    15391
Name: Masters_Degree, dtype: int64
Bachelors:
 0    50037
1    12605
Name: Bachelors_Degree, dtype: int64
Highschool:
 0    62322
1      320
Name: Highschool, dtype: int64
education:
 Master's Degree      15391
Bachelor's Degree    12601
PhD                   1703
Some College           355
Highschool             320
Name: Education, dtype: int64


Amazon       3665
Microsoft    2310
Google       1917
Facebook     1431
Apple         915
Name: company, dtype: int64

In [60]:
cleanedL4IQRcomp = cleaned[cleaned['level']=='L4'].totalyearlycompensation.quantile(0.75) - cleaned[cleaned['level']=='L4'].totalyearlycompensation.quantile(0.25)
cleanedL5IQRcomp = cleaned[cleaned['level']=='L5'].totalyearlycompensation.quantile(0.75) - cleaned[cleaned['level']=='L5'].totalyearlycompensation.quantile(0.25)
cleanedL6IQRcomp = cleaned[cleaned['level']=='L6'].totalyearlycompensation.quantile(0.75) - cleaned[cleaned['level']=='L6'].totalyearlycompensation.quantile(0.25)
cleanedL7IQRcomp = cleaned[cleaned['level']=='L7'].totalyearlycompensation.quantile(0.75) - cleaned[cleaned['level']=='L7'].totalyearlycompensation.quantile(0.25)
cleanedL8IQRcomp = cleaned[cleaned['level']=='L8'].totalyearlycompensation.quantile(0.75) - cleaned[cleaned['level']=='L8'].totalyearlycompensation.quantile(0.25)

cleanedL4upperboundcomp = cleaned[cleaned['level']=='L4'].totalyearlycompensation.quantile(0.75) + (1.5*cleanedL4IQRcomp)
cleanedL5upperboundcomp = cleaned[cleaned['level']=='L5'].totalyearlycompensation.quantile(0.75) + (1.5*cleanedL5IQRcomp)
cleanedL6upperboundcomp = cleaned[cleaned['level']=='L6'].totalyearlycompensation.quantile(0.75) + (1.5*cleanedL6IQRcomp)
cleanedL7upperboundcomp = cleaned[cleaned['level']=='L7'].totalyearlycompensation.quantile(0.75) + (1.5*cleanedL7IQRcomp)
cleanedL8upperboundcomp = cleaned[cleaned['level']=='L8'].totalyearlycompensation.quantile(0.75) + (1.5*cleanedL8IQRcomp)

In [61]:
applemerick = data[(data['company'] == 'Apple')&(data['title']=='Software Engineer')]
applemerick = applemerick.replace({'level':{'ict 2':'ICT2','ic2':'ICT2','ICT 2':'ICT2','IC2':'ICT2','New Grad (ICT2)':'ICT3'}})
applemerick = applemerick.replace({'level':{'ict3':'ICT3','ICT 3':'ICT3','IC3':'ICT3','ict3':'ICT3','ic3':'ICT3','ITC3':'ICT3','Ict3':'ICT3','Ic3':'ICT3'}})
applemerick = applemerick.replace({'level':{'ict4':'ICT4','ICT 4':'ICT4','IC4':'ICT4','ict4':'ICT4','ic4':'ICT4','ITC4':'ICT4','Ict4':'ICT4','Ic4':'ICT4','Ict 4':'ICT4','ICT-4':'ICT4','Senior Software Engineer':'ICT4'}})
applemerick = applemerick.replace({'level':{'ict5':'ICT5','ICT 5':'ICT5','IC5':'ICT5','ict5':'ICT5','ic5':'ICT5','ITC5':'ICT5','Ict5':'ICT5','Ic5':'ICT5','Ict 5':'ICT5','ICT-5':'ICT5','ICT - 5':'ICT5'}})
applemerick = applemerick.loc[(applemerick['level'] == 'ICT2') | (applemerick['level']=='ICT3') | (applemerick['level']=='ICT4') | (applemerick['level']=='ICT5')| (applemerick['level']=='ICT6')]

applemerick['level']=np.where(applemerick['level']=='ICT2','L4',applemerick['level'])

applemerick['level']=np.where((applemerick['totalyearlycompensation']<=cleanedL4upperboundcomp)&(applemerick['level']=='ICT3'),'L4',applemerick['level'])
applemerick['level']=np.where((applemerick['totalyearlycompensation']>cleanedL4upperboundcomp)&(applemerick['level']=='ICT3'),'L5',applemerick['level'])

applemerick['level']=np.where((applemerick['level']=='ICT4')&(applemerick['totalyearlycompensation']<=cleanedL5upperboundcomp),'L5',applemerick['level'])
applemerick['level']=np.where((applemerick['level']=='ICT4')&(applemerick['totalyearlycompensation']>cleanedL5upperboundcomp),'L6',applemerick['level'])

applemerick['level']=np.where((applemerick['level']=='ICT5')&(applemerick['totalyearlycompensation']<=cleanedL6upperboundcomp),'L6',applemerick['level'])
applemerick['level']=np.where((applemerick['level']=='ICT5')&(applemerick['totalyearlycompensation']>cleanedL6upperboundcomp),'L7',applemerick['level'])

applemerick['level']=np.where((applemerick['level']=='ICT6')&(applemerick['totalyearlycompensation']<=cleanedL7upperboundcomp),'L7',applemerick['level'])
applemerick['level']=np.where((applemerick['level']=='ICT6')&(applemerick['totalyearlycompensation']>cleanedL7upperboundcomp),'L8',applemerick['level'])